In [30]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

train = pd.read_csv('../Datasets/Titanic/train.csv')
test = pd.read_csv('../Datasets/Titanic/test.csv')

print train.info()
print test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null 

In [31]:
selected_features = ['Pclass','Sex','Age','Embarked','SibSp','Parch','Fare']

X_train = train[selected_features]
X_test = test[selected_features]

y_train = train['Survived']
print y_train.shape

(891,)


In [32]:
#数据预处理
print X_train['Embarked'].value_counts()
print X_test['Embarked'].value_counts()

X_train['Embarked'].fillna('S',inplace = True)
X_test['Embarked'].fillna('S',inplace = True)

X_train['Age'].fillna(X_train['Age'].mean(),inplace = True)
X_test['Age'].fillna(X_test['Age'].mean(),inplace = True)
X_test['Fare'].fillna(X_test['Fare'].mean(),inplace = True)

X_train.info()
X_test.info()

S    644
C    168
Q     77
Name: Embarked, dtype: int64
S    270
C    102
Q     46
Name: Embarked, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
Embarked    891 non-null object
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 48.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass      418 non-null int64
Sex         418 non-null object
Age         418 non-null float64
Embarked    418 non-null object
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        418 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 22.9+ KB


In [33]:
from sklearn.feature_extraction import DictVectorizer
dict_vec = DictVectorizer(sparse=False)
X_train = dict_vec.fit_transform(X_train.to_dict(orient='record'))
X_test= dict_vec.transform(X_test.to_dict(orient='record'))

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
from xgboost import XGBClassifier
xgbc = XGBClassifier()

In [34]:
from sklearn.cross_validation import cross_val_score
print cross_val_score(rfc,X_train,y_train,cv=5).mean()
print cross_val_score(xgbc,X_train,y_train,cv=5).mean()

0.796928314088972
0.81824559798311


In [35]:
rfc.fit(X_train,y_train)
rfc_y_pred = rfc.predict(X_test)
rfc_submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':rfc_y_pred})
rfc_submission.to_csv('../Datasets/Titanic/rfc_submission.csv',index = False)

xgbc.fit(X_train,y_train)
xgbc_y_pred = xgbc.predict(X_test)
xgbc_submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':xgbc_y_pred})
rfc_submission.to_csv('../Datasets/Titanic/xgbc_submission.csv',index = False)


In [36]:
from sklearn.grid_search import GridSearchCV
params = {'max_depth':range(2,7),'n_estimators':range(100,1100,200),'learning_rate':[0.05,0.1,0.25,0.5,1.0]}
xgbc_best = XGBClassifier()
gs = GridSearchCV(xgbc_best,params,n_jobs=-1,cv=5,verbose=1)
gs.fit(X_train,y_train)

Fitting 5 folds for each of 125 candidates, totalling 625 fits


[Parallel(n_jobs=-1)]: Done 625 out of 625 | elapsed:  2.1min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [100, 300, 500, 700, 900], 'learning_rate': [0.05, 0.1, 0.25, 0.5, 1.0], 'max_depth': [2, 3, 4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [37]:
print gs.best_score_
print gs.best_params_

0.835016835017
{'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 5}


In [ ]:
xgbc_best_y_pred = gs.predict(X_test)
xgbc_best_submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':xgbc_best_y_pred})
xgbc_best_submission.to_csv('../Datasets/Titanic/xgb_submission.csv',index = False)